In [1]:
with open("test3.txt") as f:
    lines = [line.strip('\n') for line in f.readlines()]
    
lines

['..........',
 '.S------7.',
 '.|F----7|.',
 '.||....||.',
 '.||....||.',
 '.|L-7F-J|.',
 '.|..||..|.',
 '.L--JL--J.',
 '..........']

In [2]:
def get_starting_point(matrix):
    for i in range(len(matrix)):
        for j in range(len(matrix[i])):
            if matrix[i][j] == 'S':
                return i,j
            
pipe_way = {
    "|" : ("north", "south"),
    "-" : ("west", "east"),
    "L" : ("north", "east"),
    "J" : ("north", "west"),
    "7" : ("west", "south"),
    "F" : ("east", "south"),
    "S" : ("east", "south","west","north"),
    "." : None,
}

direction_to_coord = {
    "north" : (-1, 0),
    "south" : (1, 0),
    "east" : (0, 1),
    "west" : (0, -1)
}

reverse_direction = {
    "north" : "south",
    "south" : "north",
    "east" : "west",
    "west" : "east"
}


coord_to_direction= {value:key for key, value in direction_to_coord.items()}

def add_tuple(tuple, touple):
    a, b = touple
    c , d = tuple
    return a+c, b+d

def get_symbol(matrix, tuple):
    a, b = tuple
    return matrix[a][b]

def set_symbol(matrix, tuple, value):
    a, b = tuple
    new_str = matrix[a][:b] + str(value) + matrix[a][b+1:]
    matrix[a] = new_str

def inbound(coord, mat):
    a, b = coord
    return a>=0 and a<len(mat) and b>=0 and b<len(mat[0])

start_point = get_starting_point(lines)
start_point

(1, 1)

In [3]:
def get_longest_path(current_coord, direction, mat):
    distances={}
    length = 1
    while inbound(current_coord, mat) and get_symbol(mat, current_coord)!='S':
        current_symbol_dirs = pipe_way[get_symbol(mat, current_coord)]
        reverse_previous_dir  = reverse_direction[direction]

        if current_symbol_dirs is not None:
            way = [symbol for symbol in current_symbol_dirs if symbol != reverse_previous_dir]

            if len(way)==1:
                distances[current_coord] = length
                length +=1
                direction = way[0]
                current_coord = add_tuple(direction_to_coord[direction], current_coord)
            else:
                return distances
        else:
            return distances
    
    if get_symbol(mat, current_coord)=='S':
        print("ouida")
        return distances


    return distances

def print_grid(matrix):
    for elem in matrix:
        print(elem)


def merge_dict(ad, bd):
    keys = [key for key in ad]
    keys += [key for key in bd if key not in keys]
    s={}
    for key in keys:
        if key not in ad:
            s[key] = bd[key]
        elif key not in bd:
            s[key] = ad[key]
        else:
            elem1, elem2 = ad[key], bd[key]
            if elem1 < elem2:
                s[key] = elem1
            else:
                s[key] = elem2
    return s 
        


In [4]:
current_position = start_point
curr_dict = {}
L = []
maxito_len = 0
path = {}
for dir, add_val in direction_to_coord.items():
    
    next_coord = add_tuple(start_point, add_val)
    sito = get_longest_path(next_coord, dir, lines)
    sito[next_coord] = 1
    if len(sito)>maxito_len:
        maxito_len = len(sito)
        path = sito
    print('---------')
L

---------
ouida
---------
ouida
---------
---------


[]

In [5]:
# curr_dict.pop((1,0))
# curr_dict.pop((0,1))
curr_dict[start_point] = 0

In [6]:
copitrix = lines[:]
for key, value in path.items():
    set_symbol(copitrix, key, value%10)
print_grid(copitrix)

..........
.S3210987.
.16789016.
.25....25.
.34....34.
.43216543.
.5..07..2.
.67898901.
..........


# inclus

In [7]:
def get_orthogonal(diro):
    ar = (diro[1], diro[0])
    br = (-diro[1], -diro[0])
    return [ar, br]

def coord_clean(cooe):
    if cooe in is_it_blocked:
        return is_it_blocked[cooe]
    return True

def is_blocked(coord, matou):    
    possible_next_coords =  [add_tuple(coord, value) for value in direction_to_coord.values()]
    if False in [inbound(coo, matou) and coord_clean(coo) for coo in possible_next_coords]:
        return False

    return True

def remove_tuple(a, b):
    x, y =a
    w, z = b
    return (x-w, y-z)

def mult_tuple(a, k):
    x, y = a
    return (x*k, y*k)

def scroll_loop_until_exit(coord, dir, hole_dir):
    two_ahead = add_tuple(coord, mult_tuple(dir, 2))
    if inbound(two_ahead) and two_ahead not in path:
        return two_ahead

    chemins = [way for way in pipe_way[get_symbol(lines, coord)] if way!=coord_to_direction(dir)]
    if len(chemins)==1:
        chem = chemins[0]
        dirms = direction_to_coord(chem)
        if dirms!=hole_dir:
            nxt_coo = add_tuple(coord, dirms)
            orthogo_dire = get_orthogonal(dirms)
            hole = None
            for dire_orth in orthogo_dire:
                voisin_orth = add_tuple(nxt_coo, dire_orth)
                if ( voisin_orth in path and abs(path[nxt_coo] - path[voisin_orth])!=1):
                    return scroll_loop_until_exit(next_coord, dirms, dire_orth)

    return None

    


def voisins_contagion(coord, already_explored, tunnel_allowed = True):
    possible_next_coords =  [add_tuple(coord, value) for value in direction_to_coord.values()]
    possible_next_coords = [next_coord for next_coord in possible_next_coords ]

    for next_coord in possible_next_coords:
        is_inside_and_new = inbound(next_coord, lines) and next_coord not in already_explored
        if_blocked = next_coord in is_it_blocked and is_it_blocked[next_coord] and next_coord not in path
        is_loop  = next_coord in path  
        hole = None


        if is_loop and tunnel_allowed:
            dir_to_get = remove_tuple(next_coord, coord)
            if coord_to_direction[dir_to_get] in pipe_way[(get_symbol(lines, next_coord))]:
                orthogo_dir = get_orthogonal(dir_to_get)
                is_loop = False
                for dire_orth in orthogo_dir:
                    voisin_orth = add_tuple(next_coord, dire_orth)
                    if ( voisin_orth in path and abs(path[next_coord] - path[voisin_orth])!=1):
                        hole = scroll_loop_until_exit(next_coord, dir_to_get, dire_orth)
    
        if hole is not None:
            is_it_blocked[next_coord] = False
            voisins_contagion(next_coord, already_explored, False)
        
        if is_inside_and_new and if_blocked:
            is_it_blocked[next_coord] = False
            voisins_contagion(next_coord, already_explored, i+1)
is_it_blocked = {}

In [8]:
for i in range(len(lines)):
    for j in range(len(lines[0])):
        if (i,j) not in is_it_blocked and (i,j) not in curr_dict:
            fct_call = is_blocked((i,j), lines)
            is_it_blocked[(i,j)] = fct_call
            if not fct_call:
                voisins_contagion((i,j), [], 0)

In [9]:
sum([1 if value else 0 for value in  is_it_blocked.values()])

0

In [10]:
tito = lambda x: "I" if x else "O"
copitrixe = lines[:]
for key, value in is_it_blocked.items():
    set_symbol(copitrixe, key, tito(value) if key not in curr_dict else "-")
for key, value in curr_dict.items():
    set_symbol(copitrixe, key, value%10)
print_grid(copitrixe)

OOOOOOOOOO
O0OOOOOOOO
OOOOOOOOOO
OOOOOOOOOO
OOOOOOOOOO
OOOOOOOOOO
OOOOOOOOOO
OOOOOOOOOO
OOOOOOOOOO
